In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
import random

In [3]:
(X_train,y_train),(X_test, y_test) = mnist.load_data()

In [4]:
X_train = X_train/255

In [5]:
X_train = X_train.reshape(-1,784)

In [6]:
X_test = (X_test/255).reshape(-1,784)

In [7]:
NOISE_DIM = 100

In [8]:
from keras.layers import LeakyReLU
# 생성자 만들기
# GAN에서 관행적으로 units=256 2의 몇제곱값 형태로 함
generator = Sequential()
generator.add(Dense(units=256 , input_dim=NOISE_DIM , activation=LeakyReLU(0.2) ))
generator.add(Dense(units=512, activation=LeakyReLU(0.2)))
generator.add(Dense(units=1024, activation=LeakyReLU(0.2)))
generator.add(Dense(units=784, activation='sigmoid'))

In [9]:
generator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
#판별자 만들기
# units을 감소시키는 방향으로 함
detector = Sequential()
detector.add(Dense(units=1024, input_dim = 784, activation= LeakyReLU(0.2)))
detector.add(Dense(units=512, activation=LeakyReLU(0.2)))
detector.add(Dense(units=256, activation=LeakyReLU(0.2)))
detector.add(Dense(units=1, activation='sigmoid'))
detector.trainable=False

In [11]:
detector.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# 식별자,판별자를 감싼 전체 신경망의 존재가 필요함
from keras.layers import Input
from keras.models import Model

total_input = Input(shape=(NOISE_DIM,))
x = generator(inputs = total_input)
# generator 신경망 만들어짐
# detector가 받게 함
total_output = detector(x)

In [13]:
model = Model(total_input, total_output)

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
EPOCHS = 10
BATCH_SIZE =128
num_of_batches = int(X_train.shape[0] / BATCH_SIZE)

for e in range(EPOCHS):
    for nob in range(num_of_batches):
        noise = np.random.normal(0,1, size=[BATCH_SIZE, NOISE_DIM])

        fake_data = generator.predict(noise)
        real_data = X_train[np.random.randint(0, X_train.shape[0],size= BATCH_SIZE)]
        
        X = np.concatenate((real_data, fake_data))
        y =np.zeros(2 * BATCH_SIZE)
        y[:BATCH_SIZE] =0.9
        
        detector.trainable =True
        d_loss = detector.train_on_batch(X, y)
        
        #탐지를 한경우
        detector.trainable =False
        noise2 = np.random.normal(0,1, size=[BATCH_SIZE, NOISE_DIM])
        y2 = np.ones(BATCH_SIZE)
        g_loss = model.train_on_batch(noise2,y2)
        
        print('EPOCH: ', e, ' BATCH: ', nob, ' g_loss:',g_loss, ' d_loss: ', d_loss)

EPOCH:  0  BATCH:  0  g_loss: [0.0008104928419925272, 1.0]  d_loss:  [3.845320224761963, 0.0]
EPOCH:  0  BATCH:  1  g_loss: [0.0008001255919225514, 1.0]  d_loss:  [3.8511099815368652, 0.0]
EPOCH:  0  BATCH:  2  g_loss: [0.0007907907711341977, 1.0]  d_loss:  [3.856325149536133, 0.0]
EPOCH:  0  BATCH:  3  g_loss: [0.00078175263479352, 1.0]  d_loss:  [3.8624610900878906, 0.0]
EPOCH:  0  BATCH:  4  g_loss: [0.0007763259345665574, 1.0]  d_loss:  [3.868183135986328, 0.0]
EPOCH:  0  BATCH:  5  g_loss: [0.0007720327703282237, 1.0]  d_loss:  [3.871040105819702, 0.0]
EPOCH:  0  BATCH:  6  g_loss: [0.000769128673709929, 1.0]  d_loss:  [3.869246482849121, 0.0]
EPOCH:  0  BATCH:  7  g_loss: [0.0007668107282370329, 1.0]  d_loss:  [3.8739185333251953, 0.0]
EPOCH:  0  BATCH:  8  g_loss: [0.0007648523896932602, 1.0]  d_loss:  [3.8752779960632324, 0.0]
EPOCH:  0  BATCH:  9  g_loss: [0.0007628387538716197, 1.0]  d_loss:  [3.8736605644226074, 0.0]
EPOCH:  0  BATCH:  10  g_loss: [0.0007607515435665846, 1.0

EPOCH:  0  BATCH:  88  g_loss: [0.0007086091209203005, 1.0]  d_loss:  [3.9121618270874023, 0.0]
EPOCH:  0  BATCH:  89  g_loss: [0.0007085047545842826, 1.0]  d_loss:  [3.9056613445281982, 0.0]
EPOCH:  0  BATCH:  90  g_loss: [0.0007083822274580598, 1.0]  d_loss:  [3.9129252433776855, 0.0]
EPOCH:  0  BATCH:  91  g_loss: [0.0007082941010594368, 1.0]  d_loss:  [3.9148387908935547, 0.0]
EPOCH:  0  BATCH:  92  g_loss: [0.000708110979758203, 1.0]  d_loss:  [3.9114608764648438, 0.0]
EPOCH:  0  BATCH:  93  g_loss: [0.0007079864153638482, 1.0]  d_loss:  [3.9125924110412598, 0.0]
EPOCH:  0  BATCH:  94  g_loss: [0.0007078201742842793, 1.0]  d_loss:  [3.9125359058380127, 0.0]
EPOCH:  0  BATCH:  95  g_loss: [0.0007076655747368932, 1.0]  d_loss:  [3.9122872352600098, 0.0]
EPOCH:  0  BATCH:  96  g_loss: [0.0007075101020745933, 1.0]  d_loss:  [3.9111781120300293, 0.0]
EPOCH:  0  BATCH:  97  g_loss: [0.000707370403688401, 1.0]  d_loss:  [3.9120125770568848, 0.0]
EPOCH:  0  BATCH:  98  g_loss: [0.00070725

EPOCH:  0  BATCH:  175  g_loss: [0.0006932458491064608, 1.0]  d_loss:  [3.9244844913482666, 0.0]
EPOCH:  0  BATCH:  176  g_loss: [0.0006929542287252843, 1.0]  d_loss:  [3.9226694107055664, 0.0]
EPOCH:  0  BATCH:  177  g_loss: [0.0006927899667061865, 1.0]  d_loss:  [3.9255752563476562, 0.0]
EPOCH:  0  BATCH:  178  g_loss: [0.0006927895010448992, 1.0]  d_loss:  [3.9220762252807617, 0.0]
EPOCH:  0  BATCH:  179  g_loss: [0.0006928318180143833, 1.0]  d_loss:  [3.923901319503784, 0.0]
EPOCH:  0  BATCH:  180  g_loss: [0.0006927156355232, 1.0]  d_loss:  [3.923595905303955, 0.0]
EPOCH:  0  BATCH:  181  g_loss: [0.000692694797180593, 1.0]  d_loss:  [3.925264835357666, 0.0]
EPOCH:  0  BATCH:  182  g_loss: [0.0006925659254193306, 1.0]  d_loss:  [3.925393581390381, 0.0]
EPOCH:  0  BATCH:  183  g_loss: [0.0006925659254193306, 1.0]  d_loss:  [3.92264461517334, 0.0]
EPOCH:  0  BATCH:  184  g_loss: [0.0006925020134076476, 1.0]  d_loss:  [3.922525644302368, 0.0]
EPOCH:  0  BATCH:  185  g_loss: [0.000692

EPOCH:  0  BATCH:  261  g_loss: [0.0006910304073244333, 1.0]  d_loss:  [3.9214282035827637, 0.0]
EPOCH:  0  BATCH:  262  g_loss: [0.0006910605006851256, 1.0]  d_loss:  [3.923649787902832, 0.0]
EPOCH:  0  BATCH:  263  g_loss: [0.0006910448428243399, 1.0]  d_loss:  [3.926511287689209, 0.0]
EPOCH:  0  BATCH:  264  g_loss: [0.0006909917574375868, 1.0]  d_loss:  [3.923692464828491, 0.0]
EPOCH:  0  BATCH:  265  g_loss: [0.0006910555530339479, 1.0]  d_loss:  [3.924952507019043, 0.0]
EPOCH:  0  BATCH:  266  g_loss: [0.0006910160882398486, 1.0]  d_loss:  [3.924450397491455, 0.0]
EPOCH:  0  BATCH:  267  g_loss: [0.0006910532247275114, 1.0]  d_loss:  [3.9267585277557373, 0.0]
EPOCH:  0  BATCH:  268  g_loss: [0.0006909932708367705, 1.0]  d_loss:  [3.9266676902770996, 0.0]
EPOCH:  0  BATCH:  269  g_loss: [0.0006910042138770223, 1.0]  d_loss:  [3.923208236694336, 0.0]
EPOCH:  0  BATCH:  270  g_loss: [0.0006909959483891726, 1.0]  d_loss:  [3.923133611679077, 0.0]
EPOCH:  0  BATCH:  271  g_loss: [0.00

EPOCH:  0  BATCH:  347  g_loss: [0.0006887782365083694, 1.0]  d_loss:  [3.9287548065185547, 0.0]
EPOCH:  0  BATCH:  348  g_loss: [0.0006887421477586031, 1.0]  d_loss:  [3.9260802268981934, 0.0]
EPOCH:  0  BATCH:  349  g_loss: [0.0006886861519888043, 1.0]  d_loss:  [3.924323081970215, 0.0]
EPOCH:  0  BATCH:  350  g_loss: [0.0006887029157951474, 1.0]  d_loss:  [3.9269165992736816, 0.0]
EPOCH:  0  BATCH:  351  g_loss: [0.000688687083311379, 1.0]  d_loss:  [3.926541328430176, 0.0]
EPOCH:  0  BATCH:  352  g_loss: [0.0006886974442750216, 1.0]  d_loss:  [3.929774761199951, 0.0]
EPOCH:  0  BATCH:  353  g_loss: [0.0006886073970235884, 1.0]  d_loss:  [3.9256441593170166, 0.0]
EPOCH:  0  BATCH:  354  g_loss: [0.0006885644979774952, 1.0]  d_loss:  [3.929020404815674, 0.0]
EPOCH:  0  BATCH:  355  g_loss: [0.0006886173505336046, 1.0]  d_loss:  [3.9306230545043945, 0.0]
EPOCH:  0  BATCH:  356  g_loss: [0.0006885975017212331, 1.0]  d_loss:  [3.9288737773895264, 0.0]
EPOCH:  0  BATCH:  357  g_loss: [0.

EPOCH:  0  BATCH:  433  g_loss: [0.0006835308158770204, 1.0]  d_loss:  [3.9307472705841064, 0.0]
EPOCH:  0  BATCH:  434  g_loss: [0.0006835264503024518, 1.0]  d_loss:  [3.9322967529296875, 0.0]
EPOCH:  0  BATCH:  435  g_loss: [0.0006835287786088884, 1.0]  d_loss:  [3.93266224861145, 0.0]
EPOCH:  0  BATCH:  436  g_loss: [0.0006835078820586205, 1.0]  d_loss:  [3.9306137561798096, 0.0]
EPOCH:  0  BATCH:  437  g_loss: [0.0006835563690401614, 1.0]  d_loss:  [3.9301023483276367, 0.0]
EPOCH:  0  BATCH:  438  g_loss: [0.0006835334934294224, 1.0]  d_loss:  [3.9312050342559814, 0.0]


KeyboardInterrupt: 

In [15]:
# 가짜 데이터 만들기
BATCH_SIZE =128
noise = np.random.normal(0,1, size=[BATCH_SIZE, NOISE_DIM])

fake_data = generator.predict(noise)

In [16]:
#실제 데이터
real_data = X_train[np.random.randint(0, X_train.shape[0],size= BATCH_SIZE)]

In [17]:
# np.concatenate => 두 데이터를 중첩
X = np.concatenate((real_data, fake_data))
y =np.zeros(2 * BATCH_SIZE)
# 1로 하면 너무 극단적이므로 0.9로 함
y[:BATCH_SIZE] =0.9

In [18]:
detector.trainable =True
# train_on_batch => 내가 지정한 만큼만 하겠다.
detector.train_on_batch(X, y)

[0.7495442628860474, 0.0]